# Imports

In [ ]:
import torch
import matplotlib.pyplot as plt
import pandas as pd
import os

In [ ]:
import matplotlib
matplotlib.rcParams['figure.facecolor'] = 'white'

In [ ]:
pd.options.display.max_columns = None

In [ ]:
from medai.datasets.iu_xray import DATASET_DIR as IU_DIR
from medai.datasets.mimic_cxr import DATASET_DIR as MIMIC_DIR

# Load utils

In [ ]:
USE_DATASET = 'iu' # 'mimic_cxr' # 'iu'
dataset_dir = MIMIC_DIR if 'mimic' in USE_DATASET else IU_DIR

## Load vocab and stuff

In [ ]:
%run ../datasets/common/constants.py
%run ../datasets/vocab/__init__.py
%run ../utils/nlp.py

In [ ]:
VOCAB = load_vocab(os.path.join(dataset_dir, 'reports'), 'v4')
REPORT_READER = ReportReader(VOCAB)
len(VOCAB)

## Load holistic chexpert

In [ ]:
fpath = os.path.join(dataset_dir, 'reports', 'reports_with_chexpert_labels.csv')
df = pd.read_csv(fpath, index_col=0)
df.replace(-1, 1, inplace=True)
df.replace(-2, 0, inplace=True)
df.head(3)

In [ ]:
REPORTS_LIST = [
    REPORT_READER.text_to_idx(report)
    for report in df['Reports']
]
len(REPORTS_LIST)

In [ ]:
def add_suffix(col):
    if col in CHEXPERT_LABELS:
        return f'{col}-gt'
    return col
df.rename(
    columns=add_suffix,
    inplace=True,
)
df.head()

In [ ]:
%run ../metrics/report_generation/chexpert.py
%run -n ../eval_report_generation_chexpert_labeler.py

In [ ]:
def _compute_metrics_vs_holistic(labels):
    columns = labels_with_suffix('gen')
    
    assert len(labels) == len(df)
    
    if isinstance(labels, torch.Tensor):
        labels = labels.cpu().numpy()
    full_df = pd.concat([
        df,
        pd.DataFrame(labels, index=df.index, columns=columns),
    ], axis=1)
    
    assert len(full_df) == len(labels)
    
    return full_df, _calculate_metrics(full_df)

# Compare light vs holistic

## Calculate light-labeler chexpert

In [ ]:
%run ../metrics/report_generation/labeler_correctness/light_labeler.py

In [ ]:
labeler = ChexpertLightLabeler(VOCAB)

In [ ]:
%%time

labels = labeler(REPORTS_LIST)
labels.shape

In [ ]:
labels[labels == -2] = 0
labels[labels == -1] = 1
labels

In [ ]:
acc, precision, recall, f1, roc_auc, pr_auc = _compute_metrics_vs_holistic(labels)
acc, precision, recall, f1, roc_auc, pr_auc

## Calculate with full-labeler

In [ ]:
%run ../metrics/report_generation/labeler_correctness/full_labeler.py
%run ../utils/nlp.py

In [ ]:
labeler = ChexpertFullLabeler(VOCAB)
labeler

In [ ]:
%%time

labels = labeler(REPORTS_LIST)
labels.shape

In [ ]:
labels[labels == -2] = 0
labels[labels == -1] = 1
labels

In [ ]:
acc, precision, recall, f1, roc_auc, pr_auc = _compute_metrics_vs_holistic(labels)
acc, precision, recall, f1, roc_auc, pr_auc

# Compare lighter vs holistic

In [ ]:
%run ../metrics/report_generation/abn_match/chexpert.py

In [ ]:
labeler = ChexpertLighterLabeler(VOCAB, device='cpu')

In [ ]:
%%time

labels = labeler(REPORTS_LIST)
labels.size()

In [ ]:
if labels.size(1) == 13:
    nf_column = torch.zeros(labels.size(0), device=labels.device).unsqueeze(-1)
    labels = torch.cat((nf_column, labels), dim=1)
labels.size()

In [ ]:
full_df, (acc, precision, recall, f1, roc_auc, pr_auc) = _compute_metrics_vs_holistic(labels)
len(acc)

In [ ]:
precision[1:], precision[1:].mean(), recall[1:], recall[1:].mean(), f1[1:], f1[1:].mean()

In [ ]:
labeler.diseases

In [ ]:
target = labeler.diseases[0]
colgt = f'{target}-gt'
colgen = f'{target}-gen'
d = full_df
d = d[((d[colgt] == 1) & (d[colgen] == 0))]
d = d[['Reports', colgt, colgen]]
print(len(d))
d.head(2)

In [ ]:
l = list(d['Reports'])
l[:10]

# Debug Lighter labeler

In [ ]:
%run ../metrics/report_generation/abn_match/chexpert.py

In [ ]:
l = ChexpertLighterLabeler(vocab)
l

In [ ]:
# reports = [
#     'heart is enlarged .',
#     'heart is not enlarged .',
#     'heart is upper limit',
# ]
sample_sentences = sentences[-50:-45]
reports_as_one = list([' '.join(sample_sentences)])
reports_as_many = list(sample_sentences)
for r in reports_as_many:
    print(r)
reports_as_one = [reader.text_to_idx(r) for r in reports_as_one]
reports_as_many = [reader.text_to_idx(r) for r in reports_as_many]

In [ ]:
l(reports_as_one)

In [ ]:
l(reports_as_many)

In [ ]:
report = reader.text_to_idx('heart size is large , no pneumothorax')
res = l([report]).tolist()[0]
list(zip(res, l.diseases))

## All reports/sentences

TODO: merge with the above code??

In [ ]:
FULL_REPORTS = False

In [ ]:
if FULL_REPORTS:
    name = 'reports_with_chexpert_labels.csv'
    TARGET_COL = 'Reports'
else:
    name = 'sentences_with_chexpert_labels.csv'
    TARGET_COL = 'sentence'
fpath = os.path.join(IU_DIR, 'reports', name)
df.replace(-2, 0, inplace=True)
df = pd.read_csv(fpath, index_col=0 if FULL_REPORTS else None)
df.head(2)

In [ ]:
texts = [reader.text_to_idx(s) for s in df[TARGET_COL]]
len(texts)

In [ ]:
%run ../metrics/report_generation/labeler_correctness/lighter_labeler/__init__.py

In [ ]:
l = ChexpertLighterLabeler(vocab, device='cpu')

In [ ]:
%%time

labels = l(texts)
labels.size()

In [ ]:
cols = [f'lighter-{d}' for d in l.diseases]
df[cols] = labels.cpu().numpy()
df.head(2)

In [ ]:
gt_labels = df[l.diseases].to_numpy().astype(np.int8)
gt_labels = torch.tensor(gt_labels, device='cpu')
gt_labels.size()

In [ ]:
tp = ((labels == 1) & (gt_labels == 1)).sum(0)
fp = ((labels == 1) & (gt_labels == 0)).sum(0)
tn = ((labels == 0) & (gt_labels == 0)).sum(0)
fn = ((labels == 0) & (gt_labels == 1)).sum(0)
tp.size(), fp.size(), tn.size(), fn.size()

In [ ]:
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * precision * recall / (precision + recall)
precision, recall, f1

In [ ]:
l.diseases

In [ ]:
l.diseases

In [ ]:
disease = l.diseases[0]
print(disease)
lighter_disease = f'lighter-{disease}'
show_cols = [TARGET_COL, 'No Finding', disease, lighter_disease]
d = df
d = d.loc[((d[disease] == 1) & (d[lighter_disease] == 0))]
d = d[show_cols].sort_values(TARGET_COL, key=lambda x: x.str.len())
print(len(d))
d.head(1)

In [ ]:
list(d[TARGET_COL].unique())

# Debug medical-labeler

With both torch and numpy arrays

In [ ]:
class NPLabeler:
    use_numpy = True
    def __call__(self, reports):
        return np.zeros((len(reports), 14))
    
class TorchLabeler:
    use_numpy = False
    def __call__(self, reports):
        return torch.zeros(len(reports), 14)

In [ ]:
TorchLabeler().__class__.__name__

In [ ]:
reports_gt = [
    [1, 2, 3, 4],
    [1, 2, 3, 5, 6, 7, 4],
]
reports_gen = [
    [1, 2, 3, 4, 6],
    [1, 2],
]

In [ ]:
%run ../metrics/report_generation/labeler_correctness/metric.py

In [ ]:
l = MedicalLabelerCorrectness(NPLabeler(), device='cuda')
l.reset()

In [ ]:
l.update((reports_gen, reports_gt))

In [ ]:
res = l.compute()
res

In [ ]:
res['acc'].mean().item()

In [ ]:
res['acc'][3].item()